In [1]:
"""
Generate positive and negative dataset.
Still not expanded.
"""
from utils.utils import generate_csvs_from_fastas
from data_preparation import generate_negative_datasets
from data_dir import epd_tata_fastas, epd_non_tata_fastas, epd_tata_csvs, epd_non_tata_csvs, epd_non_tata_generated_csvs

print("Generate positive CSV from positive EPD fastas {}".format(
    generate_csvs_from_fastas(
        epd_tata_fastas,
        epd_tata_csvs,
        [1 for f in epd_tata_fastas]
    )))

print("Generate negative csv from negative EPD fastas {}".format(
    generate_csvs_from_fastas(
        epd_non_tata_fastas,
        epd_non_tata_csvs,
        [0 for f in epd_non_tata_fastas]
    )))

from data_preparation import generate_negative_datasets
from data_dir import epd_tata_csvs, epd_non_tata_generated_csvs
print("Generate negative csv from positive EPD Data {}".format(
    generate_negative_datasets(
        epd_tata_csvs,
        epd_non_tata_generated_csvs,
        [0 for f in epd_tata_csvs]
    )))


100%|██████████| 4/4 [00:00<00:00,  9.40it/s]


Generate positive CSV from positive EPD fastas True


100%|██████████| 4/4 [00:00<00:00,  5.06it/s]


Generate negative csv from negative EPD fastas True


100%|██████████| 4/4 [00:29<00:00,  7.32s/it]

Generate negative csv from negative EPD Data True


In [1]:
from data_dir import epd_tata_csvs, epd_non_tata_csvs, epd_non_tata_generated_csvs
from utils.utils import generate_samples
import os

srcs = [epd_tata_csvs, epd_non_tata_csvs, epd_non_tata_generated_csvs]
for src in srcs:

    samples = []
    n_sample = 500
    #for f in epd_tata_csvs:
    for f in src:
        fname = os.path.basename(f)
        fname = fname.split('.')
        fextension = fname[1]
        fname = fname[0]
        fname = "{}.sample.{}.{}".format(fname, n_sample, fextension)
        dirname = os.path.dirname(f)
        fname = os.path.join(dirname, fname)
        samples.append(fname)

    generate_samples(src, samples,  n_sample=n_sample, replace=False)


 50%|█████     | 2/4 [00:00<00:00, 19.85it/s]

Sampling workspace\promoter\human_tata_min120_plus383.csv
Sampling workspace\promoter\human_tata_min256_plus255.csv
Sampling workspace\promoter\human_tata_min384_plus127.csv
Sampling workspace\promoter\human_tata_min499_plus100.csv


  0%|          | 0/4 [00:00<?, ?it/s]

Sampling workspace\promoter\human_non_tata_min120_plus383.csv


 25%|██▌       | 1/4 [00:00<00:01,  2.23it/s]

Sampling workspace\promoter\human_non_tata_min256_plus255.csv


ValueError: Cannot take a larger sample than population when 'replace=False'

In [3]:
from data_dir import epd_tata_csvs, epd_non_tata_csvs, epd_non_tata_generated_csvs
from utils.utils import generate_samples
import pandas as pd
import os

srcs = [epd_tata_csvs, epd_non_tata_csvs, epd_non_tata_generated_csvs]
for src in srcs:
    for s in src:
        df = pd.read_csv(s)
        print(f"Size of {s} is {df.shape[0]}")

Size of workspace\promoter\human_tata_min120_plus383.csv is 3065
Size of workspace\promoter\human_tata_min256_plus255.csv is 813
Size of workspace\promoter\human_tata_min384_plus127.csv is 3065
Size of workspace\promoter\human_tata_min499_plus100.csv is 3065
Size of workspace\promoter\human_non_tata_min120_plus383.csv is 26533
Size of workspace\promoter\human_non_tata_min256_plus255.csv is 141
Size of workspace\promoter\human_non_tata_min384_plus127.csv is 1065
Size of workspace\promoter\human_non_tata_min499_plus100.csv is 1386
Size of workspace\promoter\human_non_tata_generated_min120_plus383.csv is 3065
Size of workspace\promoter\human_non_tata_generated_min256_plus255.csv is 813
Size of workspace\promoter\human_non_tata_generated_min384_plus127.csv is 3065
Size of workspace\promoter\human_non_tata_generated_min499_plus100.csv is 3065


In [ ]:
"""
Generate samples for negatives.
"""
from data_preparation import generate_sample
from data_dir import epd_neg_tata_csv, epd_non_tata_csv, epd_pos_tata_csv, epd_pos_tata_csv
import os

negative_generated = os.path.join("data", "epd", "negative_tata_generated_sample.csv")
negative_epd = os.path.join("data", "epd", "negative_tata_epd_sample.csv")
positive_epd = os.path.join("data", "epd", "positive_tata_epd_sample.csv")
n_sample = 750


print(f"Generate {n_sample} sample of {epd_neg_tata_csv}: {generate_sample(epd_neg_tata_csv, negative_generated, n_sample=n_sample)}")
print(f"Generate {n_sample} sample of {epd_non_tata_csv}: {generate_sample(epd_non_tata_csv, negative_epd, n_sample=n_sample)}")
"""
Generate sample for positives.
"""
print(f"Generate {n_sample * 2} sample of {epd_pos_tata_csv}: {generate_sample(epd_pos_tata_csv, positive)}")


In [ ]:
"""
Generate kmer version for both positive and negative promoter data.
"""
from data_preparation import generate_kmer_csv
from data_dir import epd_pos_tata_csv, epd_neg_tata_csv, epd_pos_tata_kmer_csv, epd_neg_tata_kmer_csv

print("Generate kmer csv from positive data {}".format(generate_kmer_csv(epd_pos_tata_csv, epd_pos_tata_kmer_csv)))
print("Generate kmer csv from negative data {}".format(generate_kmer_csv(epd_neg_tata_csv, epd_neg_tata_kmer_csv)))

In [1]:
"""
Generate train and validation for positive and negative
"""
from data_preparation import split_and_store_csv
from data_dir import epd_pos_tata_kmer_csv, epd_neg_tata_kmer_csv, epd_pos_tata_kmer_dir, epd_neg_tata_kmer_dir, data_epd_dir

fractions = [0.9, 0.1]
split_and_store_csv(epd_pos_tata_kmer_csv, fractions, ['{}/train.positive.csv'.format(data_epd_dir), '{}/validation.positive.csv'.format(data_epd_dir)])
split_and_store_csv(epd_neg_tata_kmer_csv, fractions, ['{}/train.negative.csv'.format(data_epd_dir), '{}/validation.negative.csv'.format(data_epd_dir)])

Splitting and storing split to ./data/epd/train.positive.csv
Splitting and storing split to ./data/epd/validation.positive.csv
Splitting and storing split to ./data/epd/train.negative.csv
Splitting and storing split to ./data/epd/validation.negative.csv


True

In [2]:
"""
Merging positive and negative for each of its training and validation file.
"""
from data_dir import data_epd_dir
from data_preparation import merge_csv
for _type in ['train', 'validation']:
    pos = '{}/{}.positive.csv'.format(data_epd_dir, _type)
    neg = '{}/{}.negative.csv'.format(data_epd_dir, _type)
    target = '{}/{}.csv'.format(data_epd_dir, _type)
    merge_csv([pos, neg], target)